In [1]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


In [2]:
cd /gdrive/MyDrive/fire-detection/

/gdrive/MyDrive/fire-detection


In [3]:
pwd

'/gdrive/MyDrive/fire-detection'

In [4]:
import tensorflow as tf
import keras_preprocessing
from keras_preprocessing import image
from keras_preprocessing.image import ImageDataGenerator
TRAINING_DIR = "datasets/Train"
training_datagen = ImageDataGenerator(rescale=1./255,
zoom_range=0.15,
horizontal_flip=True,
fill_mode='nearest')
VALIDATION_DIR = "datasets/Validation"
validation_datagen = ImageDataGenerator(rescale = 1./255)
train_generator = training_datagen.flow_from_directory(
TRAINING_DIR,
target_size=(224,224),
shuffle = True,
class_mode='categorical',
batch_size = 128)
validation_generator = validation_datagen.flow_from_directory(
VALIDATION_DIR,
target_size=(224,224),
class_mode='categorical',
shuffle = True,
batch_size= 14)

Found 2780 images belonging to 2 classes.
Found 239 images belonging to 2 classes.


In [5]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Input, Dropout

from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
es = EarlyStopping(monitor='val_loss', mode='min', patience=5)
mc = ModelCheckpoint('CNN_best_model_high.h5', monitor='val_loss', mode='min', save_best_only=True)

input_tensor = Input(shape=(224, 224, 3))
base_model = InceptionV3(input_tensor=input_tensor, weights='imagenet', include_top=False)
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(2048, activation='relu')(x)
x = Dropout(0.25)(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.2)(x)
predictions = Dense(2, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)
for layer in base_model.layers:
  layer.trainable = False
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])
history = model.fit(
train_generator,
steps_per_epoch = 14,
epochs = 20,
validation_data = validation_generator,
validation_steps = 14,
callbacks=[es, mc])

Epoch 1/20
14/14 [==============================] - 502s 35s/step - loss: 19.8648 - acc: 0.6446 - val_loss: 0.1605 - val_acc: 0.9184
Epoch 2/20
14/14 [==============================] - 263s 18s/step - loss: 0.2586 - acc: 0.9053 - val_loss: 0.2271 - val_acc: 0.9184
Epoch 3/20
14/14 [==============================] - 247s 18s/step - loss: 0.1564 - acc: 0.9480 - val_loss: 0.3678 - val_acc: 0.8163
Epoch 4/20
14/14 [==============================] - 252s 18s/step - loss: 0.1527 - acc: 0.9319 - val_loss: 0.1236 - val_acc: 0.9541
Epoch 5/20
14/14 [==============================] - 245s 17s/step - loss: 0.1901 - acc: 0.9311 - val_loss: 0.1355 - val_acc: 0.9439
Epoch 6/20
14/14 [==============================] - 250s 18s/step - loss: 0.4619 - acc: 0.9002 - val_loss: 0.0944 - val_acc: 0.9592
Epoch 7/20
14/14 [==============================] - 244s 17s/step - loss: 0.1686 - acc: 0.9367 - val_loss: 0.1653 - val_acc: 0.9337
Epoch 8/20
14/14 [==============================] - 248s 18s/step - loss: 0